<a href="https://colab.research.google.com/github/Kh-Harakeh/pyomo-project-12/blob/main/12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pyomo.environ import *

# Initialize the optimization model
model = ConcreteModel()

# Define Sets
model.I = Set(initialize=['Supplier1', 'Supplier2'])  # Suppliers
model.J = Set(initialize=['Plant1', 'Plant2'])  # Plants
model.K = Set(initialize=['Retailer1', 'Retailer2'])  # Retailers
model.L = Set(initialize=['Customer1', 'Customer2'])  # Customers
model.M = Set(initialize=['CollectionCenter1', 'CollectionCenter2'])  # Collection Centers
model.D = Set(initialize=['DisassemblyCenter1', 'DisassemblyCenter2'])  # Disassembly Centers
model.R = Set(initialize=['RefurbishingCenter1', 'RefurbishingCenter2'])  # Refurbishing Centers
model.C = Set(initialize=['Component1', 'Component2', 'Component3'])  # Components
model.P = Set(initialize=[1, 2, 3])  # Time periods

# Define Parameters
model.supplier_capacity = Param(model.I, model.C, model.P, initialize=500)
model.plant_capacity = Param(model.J, model.P, initialize=1000)
model.retailer_capacity = Param(model.K, model.P, initialize=800)
model.customer_demand = Param(model.L, model.P, initialize=600)
model.collection_center_capacity = Param(model.M, model.P, initialize=700)
model.disassembly_capacity = Param(model.D, model.C, model.P, initialize=400)
model.refurbishing_capacity = Param(model.R, model.C, model.P, initialize=300)
model.transport_cost = Param(initialize=5.23)
model.collection_rate = Param(initialize=0.3)  # Percentage of products collected
model.disposal_rate = Param(initialize=0.2)  # Disposal percentage
model.refurbish_rate = Param(initialize=0.5)  # Refurbishing percentage

# Decision Variables
model.flow_suppliers_to_plants = Var(model.I, model.J, model.C, model.P, within=NonNegativeReals)
model.flow_plants_to_retailers = Var(model.J, model.K, model.P, within=NonNegativeReals)
model.flow_retailers_to_customers = Var(model.K, model.L, model.P, within=NonNegativeReals)
model.flow_customers_to_collection = Var(model.L, model.M, model.P, within=NonNegativeReals)
model.flow_collection_to_plants = Var(model.M, model.J, model.P, within=NonNegativeReals)
model.flow_collection_to_disassembly = Var(model.M, model.D, model.P, within=NonNegativeReals)
model.flow_disassembly_to_refurbishing = Var(model.D, model.R, model.C, model.P, within=NonNegativeReals)
model.flow_refurbishing_to_plants = Var(model.R, model.J, model.C, model.P, within=NonNegativeReals)
model.plant_status = Var(model.J, model.P, within=Binary)
model.retailer_status = Var(model.K, model.P, within=Binary)

# Objective Function: Minimize Total Costs
model.objective = Objective(
    expr=
    sum(model.transport_cost * (
        model.flow_suppliers_to_plants[i, j, c, p] +
        model.flow_plants_to_retailers[j, k, p] +
        model.flow_retailers_to_customers[k, l, p] +
        model.flow_customers_to_collection[l, m, p] +
        model.flow_collection_to_plants[m, j, p] +
        model.flow_collection_to_disassembly[m, d, p] +
        model.flow_disassembly_to_refurbishing[d, r, c, p] +
        model.flow_refurbishing_to_plants[r, j, c, p])
        for i in model.I for j in model.J for k in model.K for l in model.L for m in model.M
        for d in model.D for r in model.R for c in model.C for p in model.P),
    sense=minimize
)

# Constraints
# Supplier capacity
model.supplier_capacity_constraint = ConstraintList()
for i in model.I:
    for c in model.C:
        for p in model.P:
            model.supplier_capacity_constraint.add(
                sum(model.flow_suppliers_to_plants[i, j, c, p] for j in model.J) <= model.supplier_capacity[i, c, p]
            )

# Plant capacity
model.plant_capacity_constraint = ConstraintList()
for j in model.J:
    for p in model.P:
        model.plant_capacity_constraint.add(
            sum(model.flow_plants_to_retailers[j, k, p] for k in model.K) <= model.plant_capacity[j, p]
        )

# Retailer capacity
model.retailer_capacity_constraint = ConstraintList()
for k in model.K:
    for p in model.P:
        model.retailer_capacity_constraint.add(
            sum(model.flow_retailers_to_customers[k, l, p] for l in model.L) <= model.retailer_capacity[k, p]
        )

# Customer demand
model.customer_demand_constraint = ConstraintList()
for l in model.L:
    for p in model.P:
        model.customer_demand_constraint.add(
            sum(model.flow_retailers_to_customers[k, l, p] for k in model.K) == model.customer_demand[l, p]
        )

# Collection capacity
model.collection_capacity_constraint = ConstraintList()
for m in model.M:
    for p in model.P:
        model.collection_capacity_constraint.add(
            sum(model.flow_customers_to_collection[l, m, p] for l in model.L) <= model.collection_center_capacity[m, p]
        )

# Disassembly capacity
model.disassembly_capacity_constraint = ConstraintList()
for d in model.D:
    for c in model.C:
        for p in model.P:
            model.disassembly_capacity_constraint.add(
                sum(model.flow_collection_to_disassembly[m, d, p] for m in model.M) <= model.disassembly_capacity[d, c, p]
            )

# Refurbishing capacity
model.refurbishing_capacity_constraint = ConstraintList()
for r in model.R:
    for c in model.C:
        for p in model.P:
            model.refurbishing_capacity_constraint.add(
                sum(model.flow_disassembly_to_refurbishing[d, r, c, p] for d in model.D) <= model.refurbishing_capacity[r, c, p]
            )

# Solve the model
solver = SolverFactory('glpk')
result = solver.solve(model)

# Display the results
model.display()